In [ ]:
!pip install -q tensorflow tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Expand dims and normalize
train_images = np.expand_dims(train_images / 255.0, axis=-1)
test_images = np.expand_dims(test_images / 255.0, axis=-1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
baseline_model = tf.keras.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
baseline_model.fit(train_images, train_labels, epochs=2, steps_per_epoch=100, validation_split=0.1)


Epoch 1/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 410ms/step - accuracy: 0.7917 - loss: 0.7996 - val_accuracy: 0.9622 - val_loss: 0.1418
Epoch 2/2
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 283ms/step - accuracy: 0.9551 - loss: 0.1577 - val_accuracy: 0.9748 - val_loss: 0.0950


In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.grayscale_to_rgb(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
import tensorflow_hub as hub

effnet_layer = hub.KerasLayer(
    "https://tfhub.dev/google/efficientnet/b0/feature-vector/1",
    trainable=False
)


In [ ]:
class EfficientNetFeatureLayer(tf.keras.layers.Layer):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def call(self, inputs):
        return self.model(inputs)


In [ ]:
_ = effnet_layer(tf.zeros([1, 224, 224, 3]))  # Build it safely

input_tensor = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.layers.Rescaling(1./255)(input_tensor)
x = effnet_layer(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)

model_effnet = tf.keras.Model(inputs=input_tensor, outputs=output)


TypeError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_7>,) and kwargs: {} for signature: () -> Dict[['default', TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None)]].
Fallback to flat signature also failed due to: pruned(images): expected argument #0(zero-based) to be a Tensor; got KerasTensor (<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_7>).

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_7>
  • training=None

In [ ]:
model_effnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_effnet.fit(train_ds, validation_data=test_ds, epochs=3)
